In [1]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 43.2 MB/s eta 0:00:00:00:0100:01


In [2]:
import pickle
import os
import torch
import numpy as np
import faiss
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import pandas as pd

In [3]:
def save_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Đã lưu dữ liệu vào: {filename}")

def load_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    print(f"Đã tải dữ liệu từ: {filename}")
    return data

# 1. Merge

In [4]:
import glob # Để tìm file

def merge_flat_embedding_files(input_dir, output_merged_file_path, checkpoint = False):
    """
    Gộp tất cả các file flat_embeddings_*.pkl từ input_dir thành một file duy nhất.

    Args:
        input_dir (str): Thư mục chứa các file .pkl cần gộp.
        output_merged_file_path (str): Đường dẫn đầy đủ để lưu file đã gộp.
    """
    merged_chunk_data = []
    file_infos = [] # Lưu trữ thông tin từ mỗi file để tham khảo
    
    # Tìm tất cả các file .pkl theo pattern
    # Điều chỉnh pattern nếu cần, ví dụ nếu tên file có thể khác
    search_pattern = os.path.join(input_dir, "flat_embeddings_*.pkl")
    pickle_files = glob.glob(search_pattern)

    if not pickle_files:
        print(f"Không tìm thấy file .pkl nào trong thư mục: {input_dir} với pattern {search_pattern}")
        return None

    print(f"Tìm thấy {len(pickle_files)} file để gộp: {pickle_files}")

    expected_model_name = None
    expected_embedding_dim = None

    for pkl_file in pickle_files:
        print(f"Đang xử lý file: {pkl_file}")
        try:
            data = load_pickle(pkl_file)
            file_info = data.get("file_info", {})
            chunk_data_list = data.get("chunk_data", [])

            # Kiểm tra tính nhất quán (model và embedding_dim)
            current_model_name = file_info.get("model_name")
            current_embedding_dim = file_info.get("embedding_dim")

            if expected_model_name is None:
                expected_model_name = current_model_name
                expected_embedding_dim = current_embedding_dim
                print(f"Model dự kiến: {expected_model_name}, Dim dự kiến: {expected_embedding_dim}")
            elif current_model_name != expected_model_name:
                print(f"CẢNH BÁO: Model không nhất quán! File {pkl_file} có model '{current_model_name}' khác với '{expected_model_name}'. Bỏ qua file này.")
                continue # Hoặc raise error tùy bạn muốn
            elif current_embedding_dim != expected_embedding_dim:
                print(f"CẢNH BÁO: Embedding dimension không nhất quán! File {pkl_file} có dim {current_embedding_dim} khác với {expected_embedding_dim}. Bỏ qua file này.")
                continue

            if chunk_data_list:
                merged_chunk_data.extend(chunk_data_list)
                file_infos.append(file_info) # Lưu lại file_info của file này
                print(f"  Đã thêm {len(chunk_data_list)} chunks từ {pkl_file}. Tổng chunks hiện tại: {len(merged_chunk_data)}")
            else:
                print(f"  Không có chunk_data nào trong {pkl_file}.")

        except Exception as e:
            print(f"Lỗi khi xử lý file {pkl_file}: {e}. Bỏ qua file này.")
            continue
            
    if not merged_chunk_data:
        print("Không có dữ liệu chunk nào được gộp. Sẽ không tạo file merged.")
        return None

    # (Tùy chọn) Sắp xếp lại toàn bộ merged_chunk_data theo global_idx_original
    # Điều này hữu ích nếu bạn muốn một thứ tự nhất quán, mặc dù không bắt buộc cho FAISS
    # hoặc cho việc tính metrics (vì bạn sẽ nhóm theo article_id sau).
    print("Đang sắp xếp lại dữ liệu gộp theo global_idx_original...")
    merged_chunk_data.sort(key=lambda x: x["global_idx_original"])
    print("Sắp xếp hoàn tất.")
    
    # (Tùy chọn) Kiểm tra global_idx_original có bị trùng không
    global_indices_seen = set()
    duplicates_found = False
    for cd in merged_chunk_data:
        gid = cd["global_idx_original"]
        if gid in global_indices_seen:
            print(f"CẢNH BÁO: Tìm thấy global_idx_original bị trùng: {gid}")
            duplicates_found = True
            # Xử lý duplicates nếu cần, ví dụ chỉ giữ cái đầu tiên hoặc báo lỗi
        global_indices_seen.add(gid)
    if duplicates_found:
        print("LƯU Ý: Đã tìm thấy các global_idx_original bị trùng trong dữ liệu gộp.")


    # Tạo metadata cho file đã gộp
    merged_file_info = {
        "model_name": expected_model_name, # Giả sử tất cả các file gộp đều từ cùng model
        "embedding_dim": expected_embedding_dim,
        "total_chunks_merged": len(merged_chunk_data),
        "source_file_infos": file_infos # Danh sách các file_info từ các file gốc
    }

    final_merged_data = {
        "file_info": merged_file_info,
        "chunk_data": merged_chunk_data
    }
    print(f"\nĐã gộp thành công {len(merged_chunk_data)} chunks từ {len(file_infos)} files.")

    if checkpoint:
        save_pickle(final_merged_data, output_merged_file_path)
        print(f"File gộp đã được lưu tại: {output_merged_file_path}")
    
    return final_merged_data

# 2. FAISS

In [5]:
def create_faiss_index_from_merged_data(merged_data, normalize_l2=False):
    """
    Tạo FAISS index từ dữ liệu đã gộp.

    Args:
        merged_data (dict): Dữ liệu gộp chứa 'file_info' và 'chunk_data'.
        normalize_l2 (bool): Có chuẩn hóa L2 các vector trước khi thêm vào index không.
                             Hữu ích cho cosine similarity khi dùng IndexFlatIP.
                             bge-m3 embeddings thường đã được chuẩn hóa.

    Returns:
        faiss.Index: FAISS index đã được xây dựng.
        dict: Ánh xạ từ ID (global_idx_original) sang thông tin chunk.
    """
    if not merged_data or "chunk_data" not in merged_data or not merged_data["chunk_data"]:
        print("Không có dữ liệu chunk để tạo index.")
        return None, None

    chunk_data_list = merged_data["chunk_data"]
    file_info = merged_data["file_info"]

    # 1. Trích xuất embedding vectors và IDs
    embeddings = []
    ids = []
    id_to_chunk_info_map = {} # Để dễ dàng truy xuất thông tin chunk sau khi search

    for chunk_info in chunk_data_list:
        # Đảm bảo embedding_vector là numpy array và có kiểu dữ liệu phù hợp
        embedding_vector = chunk_info.get("embedding_vector")
        if not isinstance(embedding_vector, np.ndarray):
            print(f"Cảnh báo: Bỏ qua chunk có global_idx {chunk_info.get('global_idx_original')} vì embedding_vector không phải là numpy array.")
            continue
        
        embeddings.append(embedding_vector.astype('float32')) # FAISS thường làm việc với float32
        
        current_id = chunk_info["global_idx_original"]
        ids.append(current_id)
        id_to_chunk_info_map[current_id] = chunk_info # Lưu toàn bộ chunk_info

    if not embeddings:
        print("Không có embedding hợp lệ nào để thêm vào index.")
        return None, None

    embeddings_np = np.vstack(embeddings) # Chuyển list các vector thành 1 ma trận numpy
    ids_np = np.array(ids, dtype='int64') # FAISS IDs thường là int64

    # 2. Lấy dimensionality (số chiều của vector)
    # Nên lấy từ file_info nếu có, vì nó đáng tin cậy hơn
    embedding_dim = file_info.get("embedding_dim")
    if embedding_dim is None:
        embedding_dim = embeddings_np.shape[1]
        print(f"Cảnh báo: 'embedding_dim' không có trong file_info. Lấy từ shape của vector: {embedding_dim}")
    elif embedding_dim != embeddings_np.shape[1]:
        print(f"CẢNH BÁO: embedding_dim trong file_info ({embedding_dim}) "
              f"khác với shape thực tế của vector ({embeddings_np.shape[1]}). "
              f"Sử dụng shape thực tế.")
        embedding_dim = embeddings_np.shape[1]

    print(f"Số lượng embeddings: {embeddings_np.shape[0]}")
    print(f"Số chiều embedding: {embedding_dim}")

    # (Tùy chọn) Chuẩn hóa L2 nếu cần (ví dụ, để IndexFlatIP tương đương cosine similarity)
    # bge-m3 embeddings thường đã được chuẩn hóa rồi.
    if normalize_l2:
        print("Đang chuẩn hóa L2 cho các embeddings...")
        faiss.normalize_L2(embeddings_np) # Chuẩn hóa tại chỗ
        
    # 3. Tạo FAISS index
    # IndexFlatL2: Tìm kiếm dựa trên L2 distance (Euclidean). Đây là lựa chọn phổ biến.
    # IndexFlatIP: Tìm kiếm dựa trên Inner Product. Nếu vector đã chuẩn hóa L2, 
    #              thì Inner Product tỷ lệ thuận với Cosine Similarity.
    #              bge-m3 thường dùng cosine similarity, nên IndexFlatIP là lựa chọn tốt nếu đã chuẩn hóa.
    
    # Chúng ta sẽ dùng IndexFlatL2 làm ví dụ mặc định.
    # Bạn có thể đổi sang faiss.IndexFlatIP(embedding_dim) nếu muốn.
    index = faiss.IndexFlatL2(embedding_dim) 
    # index = faiss.IndexFlatIP(embedding_dim) # Nếu dùng Inner Product

    # Sử dụng IndexIDMap để có thể lưu trữ ID gốc của bạn (global_idx_original)
    # thay vì FAISS tự gán ID từ 0 đến N-1.
    index_with_ids = faiss.IndexIDMap(index)

    # 4. Thêm embeddings và IDs vào index
    print("Đang thêm embeddings vào FAISS index...")
    index_with_ids.add_with_ids(embeddings_np, ids_np)
    print(f"Đã thêm {index_with_ids.ntotal} vector vào FAISS index.")

    return index_with_ids, id_to_chunk_info_map, embedding_dim

# 3. Main function

In [6]:
# os.remove("/kaggle/working/model-0_bge-m3_fine-tuning_id_map.pkl")

In [7]:
# --- Cách sử dụng hàm gộp ---
ARRAY_EMBEDDING_MERGE_DIR = [
    ["/kaggle/input/model-0-segment-embedding-vector/TigerBear_bge-m3_fine-tuning", "/kaggle/working/model-0_bge-m3_fine-tuning.pkl"],
    ["/kaggle/input/model-1-segment-embedding-vector/TigerBear_bge-m3_fine-tuning_500_2000_512", "/kaggle/working/model-1_bge-m3_fine-tuning_500_2000_512.pkl"],
    ["/kaggle/input/model-2-segment-embedding-vector/TigerBear_bge-m3_fine-tuning_1000_full_512", "/kaggle/working/model-2_bge-m3_fine-tuning_1000_full_512.pkl"],
    ["/kaggle/input/model-3-segment-embedding-vector", "/kaggle/working/model-2_bge-m3_fine-tuning_1000_full_1000.pkl"],
    ["/kaggle/input/model-4-segment-embedding-vector/TigerBear_bge-m3_fine-tuning_1000_2000_1000", "/kaggle/working/model-4_bge-m3_fine-tuning_1000_2000_1000.pkl"]
]

for INPUT_EMBEDDINGS_DIR, OUTPUT_MERGED_FILE in ARRAY_EMBEDDING_MERGE_DIR:
    if not INPUT_EMBEDDINGS_DIR or not OUTPUT_MERGED_FILE:
        print(f"Bỏ qua mục rỗng: INPUT_DIR='{INPUT_EMBEDDINGS_DIR}', OUTPUT_FILE='{OUTPUT_MERGED_FILE}'")
        print('-----------------------------------------------------------------------')
        continue

    model_identifier = os.path.basename(OUTPUT_MERGED_FILE).replace(".pkl", "")
    print(f"===== XỬ LÝ MODEL: {model_identifier} =====")

    # if os.path.exists(OUTPUT_MERGED_FILE):
    #     print(f"File gộp {OUTPUT_MERGED_FILE} đã tồn tại. Sẽ không thực hiện gộp lại.")
    #     print('-----------------------------------------------------------------------')
    #     continue

    print(f"Xử lý: {INPUT_EMBEDDINGS_DIR} -> {OUTPUT_MERGED_FILE}")
    merged_data = merge_flat_embedding_files(INPUT_EMBEDDINGS_DIR, OUTPUT_MERGED_FILE)
    
    if merged_data and merged_data.get("chunk_data"):
        print(f"Thông tin file gộp: {merged_data['file_info']}")

        file_info = merged_data["file_info"]
        chunk_data_list = merged_data["chunk_data"]
        
        # Cập nhật lại model_identifier từ file_info nếu nó được set trong quá trình merge
        if file_info.get("model_name"):
            model_identifier = file_info["model_name"]
        print(f"Thông tin dữ liệu cho model '{model_identifier}': {file_info}")

        # Tạo FAISS index
        # bge-m3 thường được chuẩn hóa sẵn, nên normalize_l2=False là ổn.
        # Nếu bạn dùng IndexFlatIP và muốn chắc chắn về cosine similarity, đặt normalize_l2=True
        # (nhưng điều này có thể không cần thiết nếu embedding đã chuẩn hóa).
        faiss_index, id_map, embedding_dim = create_faiss_index_from_merged_data(merged_data, normalize_l2=True)

        if faiss_index:
            print(f"Đã tạo FAISS index thành công với {faiss_index.ntotal} vector.")
            
            # Lưu FAISS index (tùy chọn)
            output_faiss_index_file = OUTPUT_MERGED_FILE.replace(".pkl", "_faiss.index")
            faiss.write_index(faiss_index, output_faiss_index_file)
            print(f"Đã lưu FAISS index tại: {output_faiss_index_file}")

            # Lưu id_map (quan trọng để map lại kết quả search)
            output_id_map_file = OUTPUT_MERGED_FILE.replace(".pkl", "_id_map.pkl")
            save_pickle(id_map, output_id_map_file)
            print(f"Đã lưu id_to_chunk_info_map tại: {output_id_map_file}")


            # ----- Ví dụ cách sử dụng FAISS index để tìm kiếm -----
            if faiss_index.ntotal > 0:
                print("\n--- Ví dụ tìm kiếm FAISS ---")
                # Lấy một embedding bất kỳ từ dữ liệu để làm query (ví dụ, embedding đầu tiên)
                # Đảm bảo query_vector có cùng số chiều và kiểu dữ liệu
                first_chunk_info = merged_data["chunk_data"][0]
                query_vector_orig = first_chunk_info["embedding_vector"]
                query_vector = query_vector_orig.astype('float32').reshape(1, -1) # Phải là 2D array: (1, embedding_dim)
                
                # Nếu bạn đã normalize_l2=True khi tạo index, bạn cũng nên normalize query
                faiss.normalize_L2(query_vector) 

                k = 5 # Số lượng kết quả gần nhất muốn tìm
                print(f"Tìm kiếm {k} vector gần nhất cho vector của chunk có global_idx: {first_chunk_info['global_idx_original']}")
                
                distances, indices = faiss_index.search(query_vector, k)
                
                print("Kết quả tìm kiếm (IDs và khoảng cách):")
                for i in range(k):
                    retrieved_id = indices[0][i]
                    dist = distances[0][i]
                    
                    # Lấy thông tin chi tiết của chunk từ id_map
                    retrieved_chunk_info = id_map.get(retrieved_id)
                    if retrieved_chunk_info:
                        print(f"  Rank {i+1}: ID {retrieved_id}, Distance: {dist:.4f}")
                        print(f"    Article ID: {retrieved_chunk_info['article_id']}, "
                              f"Original Idx: {retrieved_chunk_info['original_idx_in_article']}")
                        # print(f"    Text Preview: {retrieved_chunk_info.get('text_preview', 'N/A')}") # Nếu có
                    else:
                        print(f"  Rank {i+1}: ID {retrieved_id} (không tìm thấy thông tin chi tiết), Distance: {dist:.4f}")
    else:
        print(f"Không có dữ liệu được gộp hoặc chunk_data rỗng cho {INPUT_EMBEDDINGS_DIR}.")
        
    print('-----------------------------------------------------------------------')

===== XỬ LÝ MODEL: model-0_bge-m3_fine-tuning =====
Xử lý: /kaggle/input/model-0-segment-embedding-vector/TigerBear_bge-m3_fine-tuning -> /kaggle/working/model-0_bge-m3_fine-tuning.pkl
Tìm thấy 4 file để gộp: ['/kaggle/input/model-0-segment-embedding-vector/TigerBear_bge-m3_fine-tuning/flat_embeddings_TigerBear_bge-m3_fine-tuning_213000_to_282649.pkl', '/kaggle/input/model-0-segment-embedding-vector/TigerBear_bge-m3_fine-tuning/flat_embeddings_TigerBear_bge-m3_fine-tuning_142000_to_212999.pkl', '/kaggle/input/model-0-segment-embedding-vector/TigerBear_bge-m3_fine-tuning/flat_embeddings_TigerBear_bge-m3_fine-tuning_71000_to_141999.pkl', '/kaggle/input/model-0-segment-embedding-vector/TigerBear_bge-m3_fine-tuning/flat_embeddings_TigerBear_bge-m3_fine-tuning_0_to_70999.pkl']
Đang xử lý file: /kaggle/input/model-0-segment-embedding-vector/TigerBear_bge-m3_fine-tuning/flat_embeddings_TigerBear_bge-m3_fine-tuning_213000_to_282649.pkl
Đã tải dữ liệu từ: /kaggle/input/model-0-segment-embedding

In [ ]:
from tqdm.auto import tqdm # Import tqdm

# ----- CÁC HÀM HELPER (save_pickle, load_pickle) -----
# ... (giữ nguyên)
def save_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    # print(f"Đã lưu dữ liệu vào: {filename}")

def load_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    print(f"Đã tải dữ liệu từ: {filename}")
    return data

# ----- CÁC HÀM LIÊN QUAN ĐẾN MERGE VÀ FAISS INDEX -----
# ... (load_faiss_components, load_query_embeddings giữ nguyên) ...
def load_faiss_components(base_path_pkl):
    """Tải FAISS index và id_map tương ứng."""
    faiss_index_file = base_path_pkl.replace(".pkl", "_faiss.index")
    id_map_file = base_path_pkl.replace(".pkl", "_id_map.pkl")
    faiss_index = None
    id_map = None
    embedding_dim_from_merged = None

    merged_data_file = base_path_pkl
    if os.path.exists(merged_data_file):
        try: # Thêm try-except cho việc load merged_data
            merged_data = load_pickle(merged_data_file)
            if merged_data and merged_data.get("file_info"):
                embedding_dim_from_merged = merged_data["file_info"].get("embedding_dim")
        except Exception as e:
            print(f"Lỗi khi tải file merged_data {merged_data_file} để lấy dim: {e}")
            # Vẫn tiếp tục thử tải FAISS và id_map

    if os.path.exists(faiss_index_file):
        try:
            faiss_index = faiss.read_index(faiss_index_file)
            print(f"Đã tải FAISS index từ: {faiss_index_file} ({faiss_index.ntotal} vectors)")
        except Exception as e:
            print(f"Lỗi khi tải FAISS index {faiss_index_file}: {e}")
            return None, None, None
    else:
        print(f"Không tìm thấy file FAISS index: {faiss_index_file}")
        return None, None, None

    if os.path.exists(id_map_file):
        try:
            id_map = load_pickle(id_map_file)
            if not isinstance(id_map, dict):
                print(f"Lỗi: id_map tải từ {id_map_file} không phải là dictionary.")
                return faiss_index, None, embedding_dim_from_merged 
        except Exception as e:
            print(f"Lỗi khi tải id_map {id_map_file}: {e}")
            return faiss_index, None, embedding_dim_from_merged
    else:
        print(f"Không tìm thấy file id_map: {id_map_file}")
        return faiss_index, None, embedding_dim_from_merged

    return faiss_index, id_map, embedding_dim_from_merged


def load_query_embeddings(npy_file_path):
    """Tải query embeddings từ file .npy."""
    if not os.path.exists(npy_file_path):
        print(f"Không tìm thấy file query embeddings: {npy_file_path}")
        return None
    try:
        queries = np.load(npy_file_path)
        print(f"Đã tải {queries.shape[0]} query embeddings từ {npy_file_path} (dim: {queries.shape[1]})")
        return queries.astype('float32')
    except Exception as e:
        print(f"Lỗi khi tải query embeddings từ {npy_file_path}: {e}")
        return None

# ----- CÁC HÀM TÍNH METRICS -----
# ... (precision_at_k, recall_at_k, average_precision_standard, reciprocal_rank, dcg_at_k, ndcg_at_k giữ nguyên) ...
def precision_at_k(retrieved_ids, relevant_ids, k):
    if k == 0: return 0.0
    retrieved_k = retrieved_ids[:k]
    relevant_retrieved_k = [doc_id for doc_id in retrieved_k if doc_id in relevant_ids]
    return len(relevant_retrieved_k) / k

def recall_at_k(retrieved_ids, relevant_ids, k):
    if not relevant_ids: return 0.0
    retrieved_k = retrieved_ids[:k]
    relevant_retrieved_k = [doc_id for doc_id in retrieved_k if doc_id in relevant_ids]
    return len(relevant_retrieved_k) / len(relevant_ids)

def average_precision_standard(retrieved_ids, relevant_ids):
    if not relevant_ids: return 0.0
    ap = 0.0
    hits = 0
    for i, p in enumerate(retrieved_ids):
        if p in relevant_ids:
            hits += 1
            ap += hits / (i + 1.0)
    if not relevant_ids: return 0.0 # Double check, should be covered by first if
    return ap / len(relevant_ids) if relevant_ids else 0.0

def reciprocal_rank(retrieved_ids, relevant_ids):
    for i, doc_id in enumerate(retrieved_ids):
        if doc_id in relevant_ids:
            return 1.0 / (i + 1)
    return 0.0

def dcg_at_k(retrieved_ids, relevant_ids_set, k, relevance_scores=None):
    dcg = 0.0
    for i in range(min(k, len(retrieved_ids))):
        doc_id = retrieved_ids[i]
        relevance = 0
        if relevance_scores:
            relevance = relevance_scores.get(doc_id, 0)
        elif doc_id in relevant_ids_set:
            relevance = 1
        
        if i == 0: 
            dcg += relevance
        else:
            dcg += relevance / np.log2(i + 1 + 1) 
    return dcg

def ndcg_at_k(retrieved_ids, relevant_ids_set, k, relevance_scores=None):
    dcg_val = dcg_at_k(retrieved_ids, relevant_ids_set, k, relevance_scores)
    if relevance_scores:
        ideal_relevant_docs = sorted(
            [doc_id for doc_id in relevant_ids_set if doc_id in relevance_scores],
            key=lambda doc_id: relevance_scores[doc_id],
            reverse=True
        )
    else: 
        ideal_relevant_docs = list(relevant_ids_set) 
    idcg_val = dcg_at_k(ideal_relevant_docs, relevant_ids_set, k, relevance_scores)
    return dcg_val / idcg_val if idcg_val > 0 else 0.0

# ----- HÀM ĐÁNH GIÁ CHÍNH CHO MỘT MODEL (CÓ TQDM) -----
def evaluate_model_retrieval(
    query_embeddings, 
    faiss_index, 
    id_map, 
    ground_truth, 
    k_for_search=100, 
    k_values_for_metrics=[1, 5, 10, 20, 50],
    model_name_for_tqdm="" # Thêm tên model cho tqdm
    ):
    
    num_queries = query_embeddings.shape[0]
    if num_queries == 0:
        return {}

    all_precisions = {k: [] for k in k_values_for_metrics}
    all_recalls = {k: [] for k in k_values_for_metrics}
    all_ndcgs = {k: [] for k in k_values_for_metrics}
    average_precisions = []
    reciprocal_ranks = []

    # Thêm tqdm cho vòng lặp query
    for i in tqdm(range(num_queries), desc=f"Evaluating Queries for {model_name_for_tqdm}", unit="query"):
        query_vec = query_embeddings[i:i+1]
        
        distances, retrieved_indices_from_faiss = faiss_index.search(query_vec, k_for_search)
        retrieved_global_ids = [gid for gid in retrieved_indices_from_faiss[0] if gid != -1]

        relevant_ids_for_query = ground_truth.get(i, set())

        if not relevant_ids_for_query:
            for k_val in k_values_for_metrics:
                all_precisions[k_val].append(0.0)
                all_recalls[k_val].append(0.0)
                all_ndcgs[k_val].append(0.0)
            average_precisions.append(0.0)
            reciprocal_ranks.append(0.0)
            continue

        current_ap = average_precision_standard(retrieved_global_ids, relevant_ids_for_query)
        average_precisions.append(current_ap)
        
        current_rr = reciprocal_rank(retrieved_global_ids, relevant_ids_for_query)
        reciprocal_ranks.append(current_rr)

        for k_val in k_values_for_metrics:
            p_at_k = precision_at_k(retrieved_global_ids, relevant_ids_for_query, k_val)
            all_precisions[k_val].append(p_at_k)
            
            r_at_k = recall_at_k(retrieved_global_ids, relevant_ids_for_query, k_val)
            all_recalls[k_val].append(r_at_k)

            ndcg_at_k_val = ndcg_at_k(retrieved_global_ids, relevant_ids_for_query, k_val)
            all_ndcgs[k_val].append(ndcg_at_k_val)

    results = {}
    for k_val in k_values_for_metrics:
        results[f'P@{k_val}'] = np.mean(all_precisions[k_val]) if all_precisions[k_val] else 0.0
        results[f'R@{k_val}'] = np.mean(all_recalls[k_val]) if all_recalls[k_val] else 0.0
        results[f'NDCG@{k_val}'] = np.mean(all_ndcgs[k_val]) if all_ndcgs[k_val] else 0.0
        
    results['MAP'] = np.mean(average_precisions) if average_precisions else 0.0
    results['MRR'] = np.mean(reciprocal_ranks) if reciprocal_ranks else 0.0
    
    return results

# ----- SCRIPT CHÍNH ĐỂ CHẠY ĐÁNH GIÁ (CÓ TQDM) -----

# 1. Định nghĩa đường dẫn và cấu hình
MODEL_SETUP = [
    {
        "name": "model-0_bge-m3_fine-tuning",
        "merged_pkl_path": "/kaggle/working/model-0_bge-m3_fine-tuning.pkl",
        "query_npy_path": "/kaggle/input/caption-train-segment-embedding-vector/caption_model0.npy"
    },
    {
        "name": "model-1_bge-m3_fine-tuning_500_2000_512",
        "merged_pkl_path": "/kaggle/working/model-1_bge-m3_fine-tuning_500_2000_512.pkl",
        "query_npy_path": "/kaggle/input/caption-train-segment-embedding-vector/caption_model1.npy"
    },
    {
        "name": "model-2_bge-m3_fine-tuning_1000_full_512",
        "merged_pkl_path": "/kaggle/working/model-2_bge-m3_fine-tuning_1000_full_512.pkl",
        "query_npy_path": "/kaggle/input/caption-train-segment-embedding-vector/caption_model2.npy"
    },
    {
        "name": "model-2_bge-m3_fine-tuning_1000_full_1000", # Khớp với tên file FAISS của bạn
        "merged_pkl_path": "/kaggle/working/model-2_bge-m3_fine-tuning_1000_full_1000.pkl", # Đảm bảo file này tồn tại
        "query_npy_path": "/kaggle/input/caption-train-segment-embedding-vector/caption_model3.npy" # Giả sử query cho model này
    },
    {
        "name": "model-4_bge-m3_fine-tuning_1000_2000_1000",
        "merged_pkl_path": "/kaggle/working/model-4_bge-m3_fine-tuning_1000_2000_1000.pkl",
        "query_npy_path": "/kaggle/input/caption-train-segment-embedding-vector/caption_model4.npy"
    }
]

# 2. Chuẩn bị GROUND TRUTH
# THAY THẾ BẰNG GROUND TRUTH THỰC TẾ CỦA BẠN
example_ground_truth = {}
num_example_queries = 50 
for i in range(num_example_queries):
    example_ground_truth[i] = {j for j in range(i * 10, i * 10 + 3)}
ground_truth_data = example_ground_truth

# 3. Chạy đánh giá cho từng model
all_retrieval_results = []
k_for_faiss_search = 100 
k_metrics_values = [1, 5, 10, 20, 50]

# Thêm tqdm cho vòng lặp model
for model_config in tqdm(MODEL_SETUP, desc="Processing Models", unit="model"):
    model_name = model_config["name"]
    # print(f"\n===== ĐÁNH GIÁ MODEL: {model_name} =====") # Có thể bỏ bớt print này nếu tqdm đã rõ

    faiss_index, id_map, model_embedding_dim = load_faiss_components(model_config["merged_pkl_path"])
    if faiss_index is None or id_map is None:
        print(f"Không thể tải FAISS components cho model {model_name}. Bỏ qua.")
        all_retrieval_results.append({"Model": model_name, "Error": "FAISS components missing"})
        continue

    query_embeddings_np = load_query_embeddings(model_config["query_npy_path"])
    if query_embeddings_np is None:
        print(f"Không thể tải query embeddings cho model {model_name}. Bỏ qua.")
        all_retrieval_results.append({"Model": model_name, "Error": "Query embeddings missing"})
        continue
        
    if model_embedding_dim is not None and query_embeddings_np.shape[1] != model_embedding_dim:
        print(f"CẢNH BÁO: Query embedding dim ({query_embeddings_np.shape[1]}) "
              f"không khớp với index dim ({model_embedding_dim}) cho model {model_name}.")
        all_retrieval_results.append({"Model": model_name, "Error": "Dimension mismatch"})
        continue
    
    # Kiểm tra ground truth keys
    max_gt_key = -1
    if ground_truth_data: # Chỉ kiểm tra nếu ground_truth_data không rỗng
        max_gt_key = max(ground_truth_data.keys())

    if max_gt_key >= query_embeddings_np.shape[0] :
         print(f"CẢNH BÁO: Số lượng query trong ground_truth (max key: {max_gt_key}) "
               f"vượt quá số lượng query embeddings ({query_embeddings_np.shape[0]}).")

    print(f"  Model {model_name}: Chuẩn hóa L2 cho {query_embeddings_np.shape[0]} query vectors...")
    faiss.normalize_L2(query_embeddings_np) # Quan trọng: chuẩn hóa query nếu index đã chuẩn hóa

    metrics = evaluate_model_retrieval(
        query_embeddings_np,
        faiss_index,
        id_map,
        ground_truth_data,
        k_for_search=k_for_faiss_search,
        k_values_for_metrics=k_metrics_values,
        model_name_for_tqdm=model_name # Truyền tên model cho tqdm con
    )
    
    result_entry = {"Model": model_name}
    result_entry.update(metrics)
    all_retrieval_results.append(result_entry)
    
    # print(f"  Kết quả cho {model_name}:") # Có thể bỏ bớt print này
    # for metric_name, value in metrics.items():
    #     print(f"    {metric_name}: {value:.4f}")

# 4. Hiển thị bảng so sánh
# ... (giữ nguyên)
if all_retrieval_results:
    print("\n\n===== BẢNG SO SÁNH KẾT QUẢ TRUY VẤN =====")
    results_df_retrieval = pd.DataFrame(all_retrieval_results)
    
    float_cols = [col for col in results_df_retrieval.columns if col != "Model" and col != "Error"]
    for col in float_cols:
        # Kiểm tra xem cột có chứa giá trị số không trước khi cố gắng format
        if pd.api.types.is_numeric_dtype(results_df_retrieval[col]):
             results_df_retrieval[col] = results_df_retrieval[col].apply(lambda x: f"{x:.4f}" if pd.notnull(x) else "N/A")
            
    print(results_df_retrieval.to_string())
else:
    print("\nKhông có kết quả đánh giá truy vấn nào để hiển thị.")

Processing Models:   0%|          | 0/5 [00:00<?, ?model/s]

Đã tải FAISS index từ: /kaggle/working/model-0_bge-m3_fine-tuning_faiss.index (282650 vectors)
Đã tải dữ liệu từ: /kaggle/working/model-0_bge-m3_fine-tuning_id_map.pkl
Đã tải 22040 query embeddings từ /kaggle/input/caption-train-segment-embedding-vector/caption_model0.npy (dim: 1024)
  Model model-0_bge-m3_fine-tuning: Chuẩn hóa L2 cho 22040 query vectors...


Evaluating Queries for model-0_bge-m3_fine-tuning:   0%|          | 0/22040 [00:00<?, ?query/s]